In [ ]:
# --------------------------
# Part 3 — Chronological splitting (70% train, 15% val, 15% test)
# --------------------------

WINDOW_SIZE = 120
FORECAST_HORIZON = 5
FEATURES = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']
BATCH_SIZE = 256

# Basic meta
n_rows = len(df_model)
print(f"Total rows in df_model: {n_rows}")

# Chronological split boundaries
train_end = int(np.floor(0.70 * n_rows))
val_end = int(np.floor(0.85 * n_rows))  # 70% -> 85% -> 100%

# Slice datasets (pure chronological, no shuffling)
train_df = df_model.iloc[:train_end].reset_index(drop=True)
val_df = df_model.iloc[train_end:val_end].reset_index(drop=True)
test_df = df_model.iloc[val_end:].reset_index(drop=True)


In [ ]:
# Part 4 — Scaling using only training data (StandardScaler)
# --- Prepare arrays of features for scaler and later sequence building ---

X_train_raw = train_df[FEATURES].astype(float).copy()  # DataFrame
X_val_raw = val_df[FEATURES].astype(float).copy()
X_test_raw = test_df[FEATURES].astype(float).copy()

# --- Fit scaler on training features ONLY ---
scaler = StandardScaler()
scaler.fit(X_train_raw.values)  # fit on numpy array from training set only

# Transform all splits using the training-fitted scaler
X_train_scaled = scaler.transform(X_train_raw.values)  # numpy array shape (n_train, n_features)
X_val_scaled = scaler.transform(X_val_raw.values)
X_test_scaled = scaler.transform(X_test_raw.values)

# If user later wants DataFrames back (with same columns), create them:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=FEATURES)
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=FEATURES)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=FEATURES)

# --- Label arrays for imbalance handling (these are per-row labels) ---
y_train_labels = train_df['Label'].astype(int).values
y_val_labels = val_df['Label'].astype(int).values
y_test_labels = test_df['Label'].astype(int).values

# --- Basic class distribution info (useful for imbalance handling) ---
unique_classes = np.array(sorted(df_model['Label'].dropna().unique())).astype(int)

train_class_counts = {int(c): int((y_train_labels == c).sum()) for c in unique_classes}
val_class_counts = {int(c): int((y_val_labels == c).sum()) for c in unique_classes}
test_class_counts = {int(c): int((y_test_labels == c).sum()) for c in unique_classes}

train_total = len(y_train_labels)
val_total = len(y_val_labels)
test_total = len(y_test_labels)

train_class_percent = {c: (count / train_total) * 100.0 for c, count in train_class_counts.items()}

# --- Compute initial class weights (sklearn balanced weighting) on the training labels
# This gives a starting point; given extreme imbalance you will likely combine resampling + weights.
classes_for_weights = np.array(sorted(np.unique(y_train_labels)))
class_weights = compute_class_weight(class_weight='balanced', classes=classes_for_weights, y=y_train_labels)
class_weight_dict = {int(cls): float(w) for cls, w in zip(classes_for_weights, class_weights)}

# For convenience, also expose them in the current namespace (no-op if already present)
_train_idx = (0, train_end)
_val_idx = (train_end, val_end)
_test_idx = (val_end, n_rows)

# End of this code cell — ready for Part 5.


In [ ]:
# Part 6 — Create sequences (WINDOW_SIZE -> X, FORECAST_HORIZON -> y)


# --- Helper: sliding-window sequence creator for one split ---
def create_sequences_from_split(X_split, y_split, window_size, forecast_horizon):
    """
    X_split: 2D array (n_rows, n_features)
    y_split: 1D array (n_rows,) integer labels
    Returns:
      X_seq: (n_sequences, window_size, n_features)
      y_seq: (n_sequences, forecast_horizon) ints
    """
    X_arr = np.asarray(X_split)
    y_arr = np.asarray(y_split)

    if X_arr.ndim != 2:
        raise ValueError(f"X_split must be 2D array, got shape {X_arr.shape}")
    if y_arr.ndim != 1:
        raise ValueError(f"y_split must be 1D array, got shape {y_arr.shape}")
    if X_arr.shape[0] != y_arr.shape[0]:
        raise ValueError(f"X and y must have same first-dimension length. X: {X_arr.shape[0]}, y: {y_arr.shape[0]}")

    n_rows = X_arr.shape[0]
    last_start = n_rows - window_size - forecast_horizon  # inclusive max start index
    if last_start < 0:
        # Not enough rows to construct a single sequence
        return np.empty((0, window_size, X_arr.shape[1])), np.empty((0, forecast_horizon), dtype=int)

    n_sequences = last_start + 1
    X_seq = np.empty((n_sequences, window_size, X_arr.shape[1]), dtype=X_arr.dtype)
    y_seq = np.empty((n_sequences, forecast_horizon), dtype=int)

    for i in range(n_sequences):
        X_seq[i] = X_arr[i: i + window_size]
        y_seq[i] = y_arr[i + window_size: i + window_size + forecast_horizon]

    return X_seq, y_seq


# --- Create sequences for each split ---
X_train_seq, y_train_seq = create_sequences_from_split(X_train_scaled, y_train_labels, WINDOW_SIZE, FORECAST_HORIZON)
X_val_seq, y_val_seq = create_sequences_from_split(X_val_scaled, y_val_labels, WINDOW_SIZE, FORECAST_HORIZON)
X_test_seq, y_test_seq = create_sequences_from_split(X_test_scaled, y_test_labels, WINDOW_SIZE, FORECAST_HORIZON)


# --- One-hot encode multi-step labels ---
# Determine number of classes from training labels (safe default: 3 classes {0,1,2})
classes_in_train = np.unique(y_train_seq) if y_train_seq.size > 0 else np.array([0, 1, 2])
n_classes = int(max(classes_in_train.max(), 2) + 1)  # ensures at least 3 classes (0..2)


# Convert to categorical: result shape (n_sequences, FORECAST_HORIZON, n_classes)
def one_hot_multi_step(y_seq, n_classes):
    if y_seq.size == 0:
        return np.empty((0, y_seq.shape[1], n_classes), dtype=np.float32)
    # to_categorical works on flattened array, then reshape
    flat = to_categorical(y_seq.ravel(), num_classes=n_classes)
    return flat.reshape((y_seq.shape[0], y_seq.shape[1], n_classes))


y_train_seq_cat = one_hot_multi_step(y_train_seq, n_classes)
y_val_seq_cat = one_hot_multi_step(y_val_seq, n_classes)
y_test_seq_cat = one_hot_multi_step(y_test_seq, n_classes)
